In [117]:
import pandas as pd

df.info() to check for null values

In [118]:
df = pd.read_csv('./output/MY_GE15_original.csv')
# print(df.head())

# print(df.describe())

print(df.info())

# df['constituency'] = df['constituency'].str.extract('(?<=: )(.*)') 
# df['no_of_votes'] = df['no_of_votes'].str.extract('(?<=: )(.*)')
# df['no_of_votes'] = df['no_of_votes'].str.replace(',', '').astype(float)
# df.head()
# print(df[df.isna().any(axis=1)])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 945 entries, 0 to 944
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   state           945 non-null    object
 1   constituency    945 non-null    object
 2   result          945 non-null    object
 3   candidate_name  945 non-null    object
 4   coalition       945 non-null    object
 5   no_of_votes     939 non-null    object
dtypes: object(6)
memory usage: 44.4+ KB
None


Null values in no_of_votes. Lets have a look at those rows.

In [119]:
print(df[df.isna().any(axis=1)])

     state         constituency  result               candidate_name  \
154  KEDAH  P.017: PADANG SERAI  Calon:            DATO' CIKGU AZMAN   
155  KEDAH  P.017: PADANG SERAI  Calon:        DATO SIVARAJ CHANDRAN   
156  KEDAH  P.017: PADANG SERAI  Calon:            HAMZAH ABD RAHMAN   
157  KEDAH  P.017: PADANG SERAI  Calon:              DR. SOFEE RAZAK   
158  KEDAH  P.017: PADANG SERAI  Calon:                    ANANDA AK   
159  KEDAH  P.017: PADANG SERAI  Calon:  MOHD BAKHRI BIN HAJI HASHIM   

                        coalition no_of_votes  
154            PERIKATAN NASIONAL         NaN  
155  BARISAN NASIONAL OF MALAYSIA         NaN  
156        PARTI PEJUANG TANAHAIR         NaN  
157               PAKATAN HARAPAN         NaN  
158                         BEBAS         NaN  
159           PARTI WARISAN SABAH         NaN  


They're null because that constituency's polling was postponed due to the death of one of the candidates. They will not be considered in this analysis

In [120]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 939 entries, 0 to 944
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   state           939 non-null    object
 1   constituency    939 non-null    object
 2   result          939 non-null    object
 3   candidate_name  939 non-null    object
 4   coalition       939 non-null    object
 5   no_of_votes     939 non-null    object
dtypes: object(6)
memory usage: 51.4+ KB


In [121]:
df.describe()

,state,constituency,result,candidate_name,coalition,no_of_votes
count,939,939,939,939,939,939
unique,16,221,3,938,23,904
top,SELANGOR,P.115: BATU,Telah Hilang Deposit:,RIZAL,PAKATAN HARAPAN,Jumlah Undi: 366
freq,124,10,374,2,205,3


From df.describe(), we have the right number of states (16) and the right number of constituencies (221 because we removed Padang Serai). 

2 things are fishy. First, are there really 23 coalitions? Second, there are 2 candidates named Rizal.

In [122]:
df.groupby(['coalition'])['coalition'].count().sort_values(ascending=False)

coalition
PAKATAN HARAPAN                              205
BARISAN NASIONAL OF MALAYSIA                 177
PERIKATAN NASIONAL                           148
PARTI PEJUANG TANAHAIR                       115
BEBAS                                        107
PARTI WARISAN SABAH                           51
GABUNGAN PARTI SARAWAK                        31
PARTI ISLAM SE MALAYSIA                       22
PARTI RAKYAT MALAYSIA                         16
PARTI GABUNGAN RAKYAT SABAH                   13
PARTI SARAWAK BERSATU                         10
PARTI BUMIPUTERA PERKASA MALAYSIA              9
DEMOCRATIC ACTION PARTY, MALAYSIA              8
PARTI KESEJAHTERAAN DEMOKRATIK MASYARAKAT      7
IKATAN DEMOKRATIK MALAYSIA                     6
PARTI BANGSA MALAYSIA                          5
PARTI BANSA DAYAK SARAWAK                      3
PARTI BERSATU RAKYAT SABAH                     1
PARTI PERPADUAN RAKYAT SABAH                   1
PARTI SEDAR RAKYAT SARAWAK                     1
PARTI SOSI

Okay, seems right. No typos or anything. I guess there really are 23 coalitions including independent candidates (BEBAS)

In [123]:
print(df.loc[df['candidate_name'] == 'RIZAL'])

        state                constituency                 result  \
163     KEDAH  P.018: KULIM-BANDAR BAHARU  Telah Hilang Deposit:   
192  KELANTAN        P.024: KUBANG KERIAN  Telah Hilang Deposit:   

    candidate_name               coalition       no_of_votes  
163          RIZAL  PARTI PEJUANG TANAHAIR  Jumlah Undi: 591  
192          RIZAL  PARTI PEJUANG TANAHAIR  Jumlah Undi: 687  


Lets give the Rizals their full names to tell them apart.

In [124]:
df.loc[(df['candidate_name'] == 'RIZAL') & (df['state'] == 'KEDAH'), 'candidate_name'] = 'Muhamad Yusrizal Yusuf'.upper()
df.loc[(df['candidate_name'] == 'RIZAL') & (df['state'] == 'KELANTAN'), 'candidate_name'] = 'Mohamad Rizal Razali'.upper()

Now lets clean up the constituency and no_of_votes columns

In [125]:
df[['constituency_no','constituency']]=df["constituency"].str.split(": ",expand=True) 
df['no_of_votes'] = df['no_of_votes'].str.extract('(?<=: )(.*)')
df['no_of_votes'] = df['no_of_votes'].str.replace(',', '').astype(int)

And lets standardize the result column to WON, LOST, LOST DEPOSIT


In [126]:
df['result'].unique()

array(['Dimenangi Oleh:', 'Telah Menewaskan:', 'Telah Hilang Deposit:'],
      dtype=object)

In [127]:
def format_result_column(str):
    if str=='Dimenangi Oleh:':
        return 'WON'
    elif str=='Telah Menewaskan:':
        return 'LOST'
    elif str=='Telah Hilang Deposit:':
        return 'LOST DEPOSIT'
    else:
        return "NA"

df['result'] = df['result'].apply(format_result_column)


In [128]:
df.to_csv('output/MY_GE15_cleaned.csv', index=False)